# 1. Initialisation

Importations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
matplotlib.use("TkAgg")  # Ou "Qt5Agg"
import matplotlib.animation as animation
import random
from scipy.interpolate import lagrange


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\samce\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\samce\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\samce\OneDrive\Documents\Projets\ProjetAUV\env397\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\samce\OneDrive\Documents\Projets\ProjetAUV\env397\lib\site-packages

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

Paramètres du modèle

In [ ]:
window_size = 7
distance = 5
angle = 0.3
dt = 1
noise = 0.1
resolution = 0.002
duration = 1000
beam_radius = 0
a = 0.3  # Facteur d'expansion de la spirale
b = 0.1  # Facteur de croissance du rayon

#Samuel

Initialisation des positions

In [ ]:
target_position = np.array([np.random.uniform(window_size/4, 3*window_size/4), np.random.uniform(window_size/4, 3*window_size/4)])
target_speed = np.array([np.random.uniform(-0.1, 0.1), np.random.uniform(-0.1, 0.1)])
sensor_position = np.array([window_size/2, window_size/2])
tracking_position = np.copy(sensor_position)

target_history = [target_position.copy()]
tracking_history = [tracking_position.copy()]

is_target_found = False

search_path = [(0, 0)] * duration

Initialisation du plot

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim(0, window_size)
ax.set_ylim(0, window_size)
target_line, = ax.plot([], [], 'r--', alpha=0.5, label="Trajectoire cible")
tracking_line, = ax.plot([], [], 'b--', alpha=0.5, label="Trajectoire suivi")
sensor_dot, = ax.plot([], [], 'go', markersize=8, label="Capteur")
target_dot, = ax.plot([], [], 'ro', markersize=8, label="Cible")
tracking_dot, = ax.plot([], [], 'bo', markersize=8, label="Tracking")
ax.legend()
ax.set_title("Simulation Pointing, Acquisition, and Tracking (PAT)")

Text(0.5, 1.0, 'Simulation Pointing, Acquisition, and Tracking (PAT)')

# 2. PAT

A. Outils

In [ ]:
def calculate_radius(distance, angle):
    """Calcule le rayon d'un faisceau à une certaine distance de la source"""
    radius = distance * np.tan(angle)
    return radius


def detect_target(beam_radius):
    """Teste si la cible est dans le rayon du faisceau"""
    distance = np.linalg.norm(target_position - tracking_position)
    return distance <= beam_radius

B. Calcul du parcours du faisceau

In [ ]:
def make_square_spiral_path(frame, beam_radius):
    """Calcule les positions successives d'un faisceau parcourant une spirale carrée"""
    global search_path

    point = tracking_position.copy()
    rank = frame
    search_path[rank] = point.copy()
    step = 0.5*beam_radius

    side_length = 1
    direction = 1
    while rank < duration:
        for i in range(side_length):
            rank += 1
            if rank >= duration:
                return           
            point[0] = point[0] + direction * step
            search_path[rank] = point.copy()
        for i in range(side_length):
            rank += 1
            if rank >= duration:
                return
            point[1] = point[1] + direction * step
            search_path[rank] = point.copy()
        direction *= -1
        side_length += 1


def make_circle_spiral_path(frame, beam_radius):
    """Calcule les positions successives d'un faisceau parcourant une spirale ronde"""
    global search_path

    point = tracking_position.copy()
    rank = frame
    search_path[rank] = point.copy()

    while rank < duration - 1:
        theta = a * rank  # Angle qui tourne
        r = b * rank      # Rayon qui grandit
        x = 50 + r * np.cos(theta)
        y = 50 + r * np.sin(theta)
        search_path[rank] = np.array([x, y])
        rank += 1

C. Calcul des déplacements des éléments

In [ ]:
def move_target():
    """Met à jour les position et vitesse de la cible"""
    global target_position
    global target_speed

    x, y = target_position
    vx, vy = target_speed

    new_x = x + vx*dt + random.gauss(0, noise*vx*dt)
    new_y = y + vy*dt + random.gauss(0, noise*vy*dt)
    new_vx = (new_x - x) / dt
    new_vy = (new_y - y) / dt
    
    target_position = np.array([new_x, new_y])
    target_speed = np.array([new_vx, new_vy])
    target_position = np.clip(target_position, 0, window_size)
    target_speed = np.clip(target_speed, 0, window_size)

    # Limiter la taille de l'historique pour éviter une surcharge mémoire
    if len(target_history) > 50:
        target_history.pop(0)
    target_history.append(target_position.copy())


def search_target(frame):
    """Met à jour la position d'un faisceau cherchant la cible"""
    global tracking_position, is_target_found, beam_radius
    tracking_position = np.clip(search_path[frame], 0, window_size)

    if len(tracking_history) > 50:
        tracking_history.pop(0)
    tracking_history.append(tracking_position.copy())

    if detect_target(beam_radius):
        if beam_radius <= resolution:
            is_target_found = True
        else:
            beam_radius /= 2
            make_square_spiral_path(frame, beam_radius)
            

def track_target(frame):
    """Met à jour la position d'un faisceau traquant une cible qu'il a déjà trouvé"""
    global is_target_found
    if not detect_target(beam_radius):
        is_target_found = False
        make_square_spiral_path(frame, beam_radius)    


D. Mise à jour de l'animation

In [ ]:
def update(frame):
    """Détermine l'état de la frame suivante"""
    global tracking_position, target_position

    move_target()
    if is_target_found:
        track_target(frame)        
    else :
        search_target(frame)

    # Mise à jour des données des objets graphiques
    target_x, target_y = zip(*target_history)
    tracking_x, tracking_y = zip(*tracking_history)

    target_line.set_data(target_x, target_y)
    tracking_line.set_data(tracking_x, tracking_y)
    target_dot.set_data(target_position[0], target_position[1])
    sensor_dot.set_data(sensor_position[0], sensor_position[1])
    tracking_dot.set_data(tracking_position[0], tracking_position[1])

    return target_line, target_dot, sensor_dot, tracking_line, tracking_dot,

E. Exécution

In [ ]:

beam_radius = calculate_radius(distance, angle)
make_square_spiral_path(0, beam_radius)

ani = animation.FuncAnimation(fig, update, frames=1000, interval=100, blit=True)
plt.show()

3. Génération de trajectoire 

A. Paramètres

In [ ]:
from scipy.interpolate import lagrange

n = 100       # Number of trajectories
max_deg_x = 4  # Maximum degree for x(t)
max_deg_y = 3  # Maximum degree for y(t)

B. Génération et Affichage

In [ ]:
def sample_z():
    while True:
        z = np.random.uniform(-np.sqrt(2/np.pi), np.sqrt(2/np.pi))
        p_z = np.sqrt((2/np.pi) - z**2)
        if np.random.uniform(0, np.sqrt(2/np.pi)) <= p_z:
            return z

def generate_lagrange_polynomial(max_degree):
    degree = np.random.choice(range(1, max_degree + 1))  # Randomly select the polynomial degree
    t_points = np.linspace(100, 500, degree + 1)
    values = 20 * (np.pi / 2) * np.array([sample_z() for _ in range(degree + 1)])
    return lagrange(t_points, values)

def add_noise_to_trajectories(trajectories, mu, sigma):
    noisy_trajectories = []
    for trajectory in trajectories:
        t, x_t, y_t = trajectory
        noise_x = np.random.normal(mu, sigma, size=x_t.shape)
        noise_y = np.random.normal(mu, sigma, size=y_t.shape)
        x_t_noisy = x_t + noise_x
        y_t_noisy = y_t + noise_y
        noisy_trajectories.append((t, x_t_noisy, y_t_noisy))
    return noisy_trajectories

def generate_trajectories(n, max_deg_x, max_deg_y):
    trajectories = []
    for _ in range(n):
        poly_x = generate_lagrange_polynomial(max_deg_x)
        poly_y = generate_lagrange_polynomial(max_deg_y)
        t = np.linspace(100, 500, 1000)
        x_t = poly_x(t)
        y_t = poly_y(t)
        trajectories.append((t, x_t, y_t))
    return trajectories

def plot_trajectories(trajectories):
    n = len(trajectories)
    rows = int(np.ceil(np.sqrt(n)))
    cols = int(np.ceil(n / rows))
    
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 5))
    axes = np.array(axes).flatten()
    
    for i, (t, x_t, y_t) in enumerate(trajectories):
        ax = axes[i]
        ax.plot(x_t, y_t)
        ax.set_xlabel("x(t)")
        ax.set_ylabel("y(t)")
        ax.set_title(f"Curve {i+1}")
        ax.grid()
    
    for j in range(n, len(axes)):
        fig.delaxes(axes[j])
    
    plt.tight_layout()
    plt.show()

C. Exécution

In [ ]:
trajectories = add_noise_to_trajectories(generate_trajectories(n, max_deg_x, max_deg_y), 0, 0.05)
plot_trajectories(trajectories[:10])  # Plot only 10 trajectories

NameError: name 'plt' is not defined

4. Prise en compte de la diffusion de l'eau

A. Génération de données, histogramme de valeurs et courbes COR

In [ ]:
from scipy.stats import gamma
from scipy.special import gammaln
from scipy.optimize import minimize

# Log-likelihood function to maximize
def log_likelihood(mean, alpha, yi):
    P = len(yi)
    return (P * alpha * np.log(alpha / mean)
        - P * gammaln(alpha)
        + (alpha - 1) * np.sum(np.log(yi))
        - (alpha / mean) * np.sum(yi))

# Maximum likelihood estimation of mean
def max_log_likelihood(alpha, yi):
    result = minimize(lambda m: -log_likelihood(m, alpha, yi), [1.0], method='BFGS')
    return result.x[0]

# Data generation function
def generate_data(alpha, P, n, m0, r):
    m1 = m0 * (1 + r)
    y0 = np.random.gamma(alpha, m0 / alpha, P * n)
    y1 = np.random.gamma(alpha, m1 / alpha, P * n)
    means0 = np.mean(y0.reshape(n, P), axis=1)
    means1 = np.mean(y1.reshape(n, P), axis=1)
    ml0 = np.array([max_log_likelihood(alpha, y0[i * P : (i + 1) * P]) for i in range(n)])
    ml1 = np.array([max_log_likelihood(alpha, y1[i * P : (i + 1) * P]) for i in range(n)])
    l0_0 = np.array([log_likelihood(m0, alpha, y0[i * P : (i + 1) * P]) for i in range(n)])
    l1_0 = np.array([log_likelihood(m1, alpha, y0[i * P : (i + 1) * P]) for i in range(n)])
    l0_1 = np.array([log_likelihood(m0, alpha, y1[i * P : (i + 1) * P]) for i in range(n)])
    l1_1 = np.array([log_likelihood(m1, alpha, y1[i * P : (i + 1) * P]) for i in range(n)])
    return l0_0, l1_0, l0_1, l1_1

# Histogram plotting function
def plot_histograms(l0, l1):
    diff_0 = l1[0] - l0[0]  # l1_0 - l0_0
    diff_1 = l1[1] - l0[1]  # l1_1 - l0_1
    plt.figure(figsize=(8, 6))
    plt.hist([diff_0, diff_1], bins=50, density=True, color=['blue', 'red'], alpha=0.7,
             label=["$l_{1,0} - l_{0,0}$", "$l_{1,1} - l_{0,1}$"])
    plt.axvline(0, color='black', linestyle='--', linewidth=1, label='Zero')
    plt.xlabel("Value")
    plt.ylabel("Frequency")
    plt.grid()
    plt.legend()
    plt.show()

# ROC curve plotting function
def plot_roc(diff_0, diff_1, num_thresholds=1000):
    thresholds = np.linspace(min(diff_0.min(), diff_1.min()), max(diff_0.max(), diff_1.max()), num_thresholds)
    p_fa = [(diff_0 >= t).mean() for t in thresholds]
    p_d = [(diff_1 >= t).mean() for t in thresholds]
    plt.figure(figsize=(8, 6))
    plt.plot(p_fa, p_d, color='blue', label='ROC Curve')
    plt.xscale('log')
    plt.xlabel(r'$ P_{fa} $')
    plt.ylim([0, 1.05])
    plt.xlim([1e-5, 1])
    plt.ylabel(r'$ P_{de} $')
    plt.grid()
    plt.legend()
    plt.show()

B. Test

In [ ]:
# Generate data and plot results
l0_0, l1_0, l0_1, l1_1 = generate_data(2, 10, 1000000, 5, 0.5)
plot_histograms([l0_0, l0_1], [l1_0, l1_1])
plot_roc(l1_0 - l0_0, l1_1 - l0_1)